In [1]:
# 导入相关库
import os
import pandas as pd
from tqdm.autonotebook import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
os.environ['PYTHONHASHSEED'] = '0'
# 显卡使用（如没显卡需要注释掉）
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"
np.random.seed(1024)
rn.seed(1024)
tf.set_random_seed(1024)
path="data/"
os.listdir("data/")

/opt/conda/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
Using TensorFlow backend.


['set_series_1.csv',
 'user_app_actived.csv',
 'age_train.csv',
 'age_test.csv',
 'app_info.csv',
 'train_data.csv',
 'user_basic_info.csv',
 'user_app_usage.csv',
 'user_behavior_info.csv']

In [2]:
# 读入数据（需加速）
def get_age_data():
    train_data = pd.read_csv(path + 'age_train.csv', header=None)
    test_data = pd.read_csv(path + 'age_test.csv', header=None)
    data = pd.concat([train_data, test_data], axis=0, sort=False).fillna(-1)
    data.columns = ['uId', 'age_group']
    return data

def get_user_app_actived():
    data = pd.read_csv(path + 'user_app_actived.csv', header=None)
    data.columns = ['uId', 'appId']
    return data

def get_user_behavior_info():
    data = pd.read_csv(path + 'user_behavior_info.csv', header=None)
    data.columns = ['uId', 'bootTimes', 'AFuncTimes', 'BFuncTimes', 'CFuncTimes',
                   'DFuncTimes', 'EFuncTimes', 'FFuncTimes', 'FFuncSum']
    return data

def get_user_basic_info():
    data = pd.read_csv(path + 'user_basic_info.csv', header=None)
    data.columns = ['uId', 'gender', 'city', 'prodName', 'ramCapacity', 
                   'ramLeftRation', 'romCapacity', 'romLeftRation', 'color',
                   'fontSize', 'ct', 'carrier', 'os']
    return data

def get_app_info():
    data = pd.read_csv(path + 'app_info.csv', header=None)
    data.columns = ['appId', 'category']
    return data

# 测试的时候用True
# 提特征改用False
def get_user_app_usage(less_data=False):
    if less_data:
        reader = pd.read_csv(path + 'user_app_usage.csv', chunksize=2000000)
        for i in reader:
            data = i
            break
    else:
        data = pd.read_csv(path + 'user_app_usage.csv', header=None)
    data.columns = ['uId', 'appId', 'duration', 'times', 'use_date']
    return data

def get_time_usage():
    data = pd.read_csv(path + 'set_series_1.csv', header=None)
    data.columns = ['uId', 'timesUsage']
    return data

def get_usage_set():
    data = pd.read_csv(path + 'nurbs_set.csv', header=None)
    data.columns = ['uId', 'timeCount']
    return data

In [3]:
# 需要用到的函数
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


from keras.engine.topology import Layer
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
### 读入数据(想要五输入)
id_label = get_age_data()
active_data = get_user_app_actived()
data = pd.merge(id_label, active_data, on='uId', how='left')
data_info = get_app_info()
cat_dict = dict(zip(data_info['appId'], data_info['category']))
def get_review_data(row):
    review = ''
    try:
        app_list = row['appId'].split('#')
    except Exception:
        app_list = ['0']
        print(row['appId'])
    for i in app_list:
        try:
            review += cat_dict[i] + ' '
        except Exception:
            review += '未知' + ' ' 
    return review
data['appInfo'] = data.apply(lambda row:get_review_data(row), axis=1)
time_usage_data = get_time_usage()
data = pd.merge(data, time_usage_data, on='uId', how='left')
data = data.fillna('未知')
data.head(5)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
del id_label, active_data, data_info, cat_dict

In [ ]:
### Tokenizer 序列化文本
def set_tokenizer(docs, split_char=' ', max_len=100):
    '''
    输入
    docs:文本列表
    split_char:按什么字符切割
    max_len:截取的最大长度
    
    输出
    X:序列化后的数据
    word_index:文本和数字对应的索引
    '''
    tokenizer = Tokenizer(lower=False, char_level=False, split=split_char)
    tokenizer.fit_on_texts(docs)
    X = tokenizer.texts_to_sequences(docs)
    maxlen = max_len
    X = pad_sequences(X, maxlen=maxlen, value=0)
    word_index=tokenizer.word_index
    return X, word_index

### 做embedding 这里采用word2vec 可以换成其他例如（glove词向量）
def trian_save_word2vec(docs, embed_size=300, save_name='w2v.txt', split_char=' '):
    '''
    输入
    docs:输入的文本列表
    embed_size:embed长度
    save_name:保存的word2vec位置
    
    输出
    w2v:返回的模型
    '''
    input_docs = []
    for i in docs:
        input_docs.append(i.split(split_char))
    logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(input_docs, size=embed_size, sg=1, window=8, seed=1017, workers=24, min_count=1, iter=10)
    w2v.wv.save_word2vec_format(save_name)
    print("w2v model done")
    return w2v

# 得到embedding矩阵
def get_embedding_matrix(word_index, embed_size=300, Emed_path="w2v_300.txt"):
    embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
        Emed_path, binary=False)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    
    print("null cnt",count)
    return embedding_matrix

In [ ]:
# appId 90
# timesUsage 600
text_1_list = list(data['appId'])
text_3_list = list(data['timesUsage'])

del data['appId']
del data['timesUsage']

print('开始序列化')
x1, index_1 = set_tokenizer(text_1_list, split_char='#', max_len=90)
x3, index_3 = set_tokenizer(text_3_list, split_char=' ', max_len=600)
print('序列化完成')
gc.collect()

np.save('tmp/x1.npy', x1)
np.save('tmp/x3.npy', x3)
np.save('tmp/x1.npy', index_1)
np.save('tmp/x3.npy', index_3)

# 值得提醒的是这个保存方法是采用w2v.wv.save_word2vec_format
# 因此你如果载入自己训练模型的时候，需要载入后再按照这个函数来保存再在emed_path中输入
trian_save_word2vec(text_1_list, save_name='w2v_model/cate_w2v_300.txt', split_char='#')
gc.collect()
trian_save_word2vec(text_3_list, save_name='w2v_model/w2v_300.txt', split_char=' ')
gc.collect()

# 得到emb矩阵
emb1 = get_embedding_matrix(index_1, Emed_path='w2v_model/cate_w2v_300.txt')
emb3 = get_embedding_matrix(index_3, Emed_path='w2v_model/w2v_300.txt')
gc.collect()

In [ ]:
# 将feature作为输入进行处理(这个feature一定要替换掉nan与inf)
# 这个feature的顺序一定要对应之前的feature的顺序，保证每条是对应的
f1 = pd.read_csv('feature/f1.csv')
f2 = pd.read_csv('feature/f2.csv')
f3 = pd.read_csv('feature/f3.csv')
f4 = pd.read_csv('feature/f4.csv')
f5 = pd.read_csv('feature/f5.csv')

feature = pd.concat([f1, f2, f3, f4, f5], axis=1, sort=False)
feature = feature.fillna(-1)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(feature)
hin_feature = ss.transform(feature)
num_feature_input = hin_feature.shape[1]

In [ ]:
# 区分开train和valid,test
# 这里是假设三输入
train_data = data[data['age_group']!=-1]
train_input_1 = x1[:len(train_data)]
test_input_1 = x1[len(train_data):]
train_input_3 = x3[:len(train_data)]
test_input_3 = x3[len(train_data):]
train_input_5 = hin_feature[:len(train_data)]
test_input_5 = hin_feature[len(train_data):]
label = to_categorical(train_data['age_group'] - 1)

In [ ]:
from keras.initializers import *

def model_conv(emb1, emb3, num_feature_input):
    '''
    注意这个inputs
    seq1、seq2分别是两个输入
    hin是feature层输入
    是否做emb可选可不选，
    这个就是我们之前训练已经得到的用于embedding的（embedding_matrix1， embedding_matrix2）
    '''
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=90,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=600,
        trainable=False
    )
    
    
    seq1 = Input(shape=(90,))
    seq3 = Input(shape=(600,))    
    
    x1 = emb_layer_1(seq1)
    x3 = emb_layer_3(seq3)
    
    sdrop=SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x3 = sdrop(x3)
    
    x = Dropout(0.2)(Bidirectional(CuDNNLSTM(200, return_sequences=True))(x1))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    merged_1_avg = Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,))(semantic)
    
    x = Dropout(0.2)(Bidirectional(CuDNNLSTM(200, return_sequences=True))(x3))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_3 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    merged_3_avg = Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,))(semantic)
    
    hin = Input(shape=(num_feature_input, ))
    htime = Dense(16, activation='relu')(hin)
    
    x = concatenate([merged_1, merged_3, merged_1_avg, merged_3_avg, htime])
    
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(x))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    pred = Dense(6, activation='softmax')(x)
    model = Model(inputs=[seq1, seq3, hin], outputs=pred)
    from keras.utils import multi_gpu_model
    model = multi_gpu_model(model, 2)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamW(lr=0.001,weight_decay=0.08,),metrics=["accuracy"])
    return model
gc.collect()

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=1017, shuffle=True)
sub = np.zeros((test_input_5.shape[0], 6))
oof_pred = np.zeros((train_input_5.shape[0], 6))
score = []
count = 0
if not os.path.exists("model"):
    os.mkdir("model")

for i, (train_index, test_index) in enumerate(skf.split(train_input_5, train_data['age_group'])):
    print("FOLD | ", count+1)
    print("###"*35)
    gc.collect()
    filepath = "model/nn_v1_%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(
        monitor='val_acc', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_acc', min_delta=0.0001, patience=5, verbose=1, mode='max')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    model_age = model_conv(emb1, emb3, num_feature_input)
    if count==0:model_age.summary()
    x1_tr, x1_va = np.array(train_input_1)[train_index], np.array(train_input_1)[test_index]    
    x3_tr, x3_va = np.array(train_input_3)[train_index], np.array(train_input_3)[test_index]
    x5_tr, x5_va = np.array(train_input_5)[train_index], np.array(train_input_5)[test_index]
    y_tr, y_va = label[train_index], label[test_index]
    
    hist = model_age.fit([x1_tr, x3_tr, x5_tr],
                         y_tr, batch_size=4096, epochs=50, 
                         validation_data=([x1_va, x3_va, x5_va], y_va),
                         callbacks=callbacks, verbose=1, shuffle=True)

    model_age.load_weights(filepath)
    oof_pred[test_index] = model_age.predict([x1_va, x3_va, x5_va],batch_size=2048,verbose=1)
    sub += model_age.predict([test_input_1, test_input_3, test_input_5],batch_size=2048,verbose=1)/skf.n_splits
    score.append(np.max(hist.history['val_acc']))
    count += 1
print('acc:', np.mean(score))

In [ ]:
test = data[data['age_group'] == -1]
submit = test[['uId']]
submit.columns = ['id']
submit['label'] = sub.argmax(1)+1
if not os.path.exists("result"):
    os.mkdir("result")
submit.to_csv("./result/submission.csv",index=False)

In [ ]:
oof = np.concatenate((oof_pred,sub))
oof = pd.DataFrame(oof)
oof.columns = [str(i+1) for i in range(6)]
oof['id'] = pd.concat([train_data[['uId']],test[['uId']]])['uId'].values
oof.to_csv("./result/v1_test.csv",index=False)

In [ ]:
!nvidia-smi